In [ ]:
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from enum import Enum, auto
from random import uniform, random
from math import cos, sin, pi, inf, sqrt
import os

radius = 0.075
dt = 0.01
half = 1
speed = 1
time_to_recover = 0.05
buff = 1

class State(Enum):
    Susceptible = auto()
    Infected = auto()
    Recovered = auto()

class Person:
    
    def __init__(self,co,state,standstill):
        
        # 93_self.persons.append(Person(co,state,standstill))
        self.co = co
        self.coords = np.array([self.co])    # 座標列[co_0,co_1,co_2,,]
        self.arg = uniform(-pi, pi)
        self.standstill = standstill    # 停止　True-or-Furse
        if self.standstill:
            vel = np.array([0.0, 0.0])    # 停止　True=vel[x_0,y_0]
        else:
            vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
        self.vel = vel        # [0,0]_or_[x_arg,y_arg]
        self.state = state    # S,I,R
        self.infection_start = inf    # I_start_i(時間index)
        
    def move(self):
        if self.standstill:
            self.vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
            
class Population:
    
    def __init__(self,number,comeback,p=None):
        
        # 230_Population(numbe =10   ,comeback =True,  p =0.5)
        self.number = number
        self.R0 = (radius * (sqrt(2) * speed) * time_to_recover * self.number) / (half ** 2)
        print("R0: %f" % self.R0)
        self.persons = []
        self.p = p
        init_coords = []
        if self.p is not None:
            self.Re = (1.0 - self.p) * (1.0 + (sqrt(2) - 1) * self.p) * self.R0
            print("Re: %f" % self.Re)
        for i in range(self.number):    # [0,1,2,,]順に代入
            if i == 0:
                init_co = np.array([0.0, 0.0])
                infected = Person(co=init_co,state=State.Infected,standstill=False)
                #                  座標     ,感染 I              ,停止ではない
                infected.infection_start = 0    # I_start_i=0_感染源
                self.persons.append(infected)    # object_追記
            else:
                whi=0
                while True:
                    init_co = np.array([uniform(-half, half), uniform(-half, half)])
                    appropriate = True
                    foi=0
                    for pre_init_co in init_coords:
                        dist = np.linalg.norm(init_co - pre_init_co, ord=2)
                        if dist <= 3 * radius:
                            print("72___dist-3*radius=",dist-3*radius)
                            appropriate = False
                            
                        if appropriate == False:
                            print("76_i-whi-for_break=",i,"-",whi,"-",foi,"\n")
                            break
                            
                        foi += 1
                        
                    if appropriate:    # False,は,while(座標作,重り)繰返し
                        break
                        
                    whi += 1
                    
                if self.p is None:
                    standstill = False
                elif random() < self.p:
                    standstill = True
                else:
                    standstill = False
                self.persons.append(Person(co=init_co,
                                           state=State.Susceptible,
                                           standstill=standstill))
                print("95_i ,whi, persons)= ",i,",",whi,",",self.persons)
                
            init_coords.append(init_co)
            # print("--- 98_i,init_coords= ",i,",",init_coords,"\n")
            
        # print("100_vars(self.persons[3])=",vars(self.persons[3]))


    def simulate_sir(self,threshold=2):    # 閾値=2
        
        i = 0                        # 時間の index.
        exceeded_threshold = False    # 感染者数が閾値（threshold）を超えた(exceeded)か否か.
        comebacked = False    # 行動制限を解除したか否か.
        subsided = None    # 感染が収束した時の時間のインデックス.
        
        # 感染収束 subsided + バッファ =時間インデックス ⇒ def_simulate_sir
        
        while True:
            # 感染始 I⇒回復時間 time_to_recover⇒免疫獲得 R 動再開
            # 正方形 Box 壁に当り 反射
            for person in self.persons:    # [object列,0,1,2,,]順に代入
                if (i - person.infection_start) * dt > time_to_recover:
                    # time_index - object_要素(感染始time_index)*0.01> 0.05
                    person.state = State.Recovered    # 要素state_I⇒R
                    
                # vel[x,y]×dt=[xdt,ydt]=Δvel ,座標値 co+Δvel
                person.co += dt * person.vel
                # ｘ+Δxで、壁反射.
                if abs(person.co[0]) > half:
                    person.co[0] = np.sign(person.co[0]) * half
                    person.vel[0] *= -1
                # ｙ+Δｙで、壁反射.
                if abs(person.co[1]) > half:
                    person.co[1] = np.sign(person.co[1]) * half
                    person.vel[1] *= -1
                    
            indices = list(range(self.number))
            print("indices=",indices)
            whi=0
            while len(indices) > 0:
                k = indices.pop(0)
                person = self.persons[k]
                for l in indices:
                    other = self.persons[l]
                    vec = person.co - other.co
                    dist = np.linalg.norm(vec, ord=2)
                    # person と other の距離が 2 *（半径）より小さいとき接触したと判定.
                    # 簡単のため 3 人以上の同時接触は考えない.
                    if 0 < dist < 2 * radius:
                        normal = vec / dist
                        # person と other が重ならないようにするための補正（correction）.
                        correction = (radius - (dist / 2)) * normal
                        person.co += correction
                        other.co -= correction
                        # 物理的な完全弾性衝突の場合は -= np.dot(person.vel - other.vel, normal) * normal.
                        person.vel -= (2 * np.dot(person.vel, normal)) * normal
                        # 物理的な完全弾性衝突の場合は -= np.dot(other.vel - person.vel, normal) * normal.
                        other.vel -= (2 * np.dot(other.vel, normal)) * normal
                        indices.remove(l)
                        # 感染状態を必要であれば変化させる.
                        if person.state == State.Infected and other.state == State.Susceptible:
                            other.state = State.Infected
                            other.infection_start = i
                        elif person.state == State.Susceptible and other.state == State.Infected:
                            person.state = State.Infected
                            person.infection_start = i
                        break
                        
                    # if_
                # for_in
                print("whi=",whi)
                whi+=1
                
            # while_時間の index を一つ進める
            peint("時間index, i=",i)
            i += 1
            
            I = 0    # 総感染者数
            S = 0    # 総未感染者数
            R = 0    # 総免疫獲得者数
            
            for person in self.persons:
                
                # 各人間に対し, 現在の座標を追加記憶させる.
                # 2次元縦方向(vertical),axis=0,行方向連結
                person.coords = np.vstack([person.coords, person.co])
                print("182_行方向連結_vstack(coords←co)=",person.coords)
                
                if person.state == State.Infected:
                    I += 1
                elif person.state == State.Susceptible:
                    S += 1
                elif person.state == State.Recovered:
                    R += 1
                    
            # もし総感染者数 I が感染者数の閾値（threshold）を
            # 初めて超えた場合, exceeded_threshold を真とする.
            
            if threshold < I and not exceeded_threshold:
                exceeded_threshold = True
                
            # 行動制限を解除する場合（self.comeback が真の場合）,
            # もしまだ行動制限を解除しておらず（comebacked が偽）,
            # 総感染者数 I が感染者数の閾値（threshold）を既に超えており（exceeded_threshold が真）
            # その後総感染者数 I が感染者数の閾値（threshold）を下回った場合,
            # 停止させている人間は全て動かす.
            
            if I <= threshold and self.comeback and not comebacked and exceeded_threshold:
                for person in self.persons:
                    person.move()
                comebacked = True
                
            # 総感染者数が 0 つまり感染が収束した場合,
            # 感染収束時の時間の index を subsided に記憶させる.
            
            if I == 0 and subsided is None:
                subsided = i
            if comebacked:
                print("Simulating SIR %d (I: %d, S: %d, R: %d) (Comebacked)" % (i, I, S, R))
            else:
                print("Simulating SIR %d (I: %d, S: %d, R: %d)" % (i, I, S, R))
                
            # それぞれの数値計算結果を記憶.
            self.times = np.append(self.times, i * dt)
            self.zeros = np.append(self.zeros, 0)
            self.I = np.append(self.I, I)
            self.S = np.append(self.S, S)
            self.R = np.append(self.R, R)
            
            # 感染収束時の時間の index が subsided に記憶されており,
            # 時間の index（= i）= subsided +バッファ
            #  ⇒ シミュレーションを終了 break
            if subsided is not None and i == subsided + buff:
                self.end = i
                break


if __name__ == "__main__":
    population = Population(number=10 ,comeback=True,p=0.5)
    population.simulate_sir()
    # population.animate()
    